![Credit card being held in hand](credit_card.jpg)

Commercial banks receive _a lot_ of applications for credit cards. Many of them get rejected for many reasons, like high loan balances, low income levels, or too many inquiries on an individual's credit report, for example. Manually analyzing these applications is mundane, error-prone, and time-consuming (and time is money!). Luckily, this task can be automated with the power of machine learning and pretty much every commercial bank does so nowadays. In this workbook, you will build an automatic credit card approval predictor using machine learning techniques, just like real banks do.

### The Data

The data is a small subset of the Credit Card Approval dataset from the UCI Machine Learning Repository showing the credit card applications a bank receives. This dataset has been loaded as a `pandas` DataFrame called `cc_apps`. The last column in the dataset is the target value.

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

import streamlit as st


In [2]:

# Load the dataset
data = pd.read_csv("cc_approvals.data", header=None) 
data.head()

0      1      2  3  4  5  6     7  8  9   10 11   12 13
0  b  30.83  0.000  u  g  w  v  1.25  t  t   1  g    0  +
1  a  58.67  4.460  u  g  q  h  3.04  t  t   6  g  560  +
2  a  24.50  0.500  u  g  q  h  1.50  t  f   0  g  824  +
3  b  27.83  1.540  u  g  w  v  3.75  t  t   5  g    3  +
4  b  20.17  5.625  u  g  w  v  1.71  t  f   0  s    0  +

In [3]:
data.shape

(690, 14)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       690 non-null    object 
 1   1       690 non-null    object 
 2   2       690 non-null    float64
 3   3       690 non-null    object 
 4   4       690 non-null    object 
 5   5       690 non-null    object 
 6   6       690 non-null    object 
 7   7       690 non-null    float64
 8   8       690 non-null    object 
 9   9       690 non-null    object 
 10  10      690 non-null    int64  
 11  11      690 non-null    object 
 12  12      690 non-null    int64  
 13  13      690 non-null    object 
dtypes: float64(2), int64(2), object(10)
memory usage: 75.6+ KB


In [5]:
data.nunique()

0       3
1     350
2     215
3       4
4       4
5      15
6      10
7     132
8       2
9       2
10     23
11      3
12    240
13      2
dtype: int64

### Understanding the Dataset

The dataset has 690 rows and 14 columns. Here's a summary of the data:

- **Data Types**: There are various data types including `float64`, `int64`, and `object`.
- **Missing Values**: We will need to check for and handle any missing or erroneous values.
- **Target Variable**: The last column (index 13) is the target variable indicating whether the credit card application was approved (`+`) or not (`-`).

### Column Overview

The columns are not named, but for better understanding, let's assign temporary names based on the structure we see. The dataset appears to have both categorical and numerical features.

In [6]:
# Assign column names
column_names = [
    'Gender', 'Age', 'Debt', 'Marital_Status', 'Bank_Customer', 'Education_Level', 
    'Ethnicity', 'Years_Employed', 'Prior_Default', 'Employed', 'Credit_Score', 
    'Drivers_License', 'Citizen', 'Approval_Status'
]
data.columns = column_names

# Display unique values for each column to understand their nature
unique_values = {col: data[col].unique() for col in data.columns}

unique_values

{'Gender': array(['b', 'a', '?'], dtype=object),
 'Age': array(['30.83', '58.67', '24.50', '27.83', '20.17', '32.08', '33.17',
        '22.92', '54.42', '42.50', '22.08', '29.92', '38.25', '48.08',
        '45.83', '36.67', '28.25', '23.25', '21.83', '19.17', '25.00',
        '47.75', '27.42', '41.17', '15.83', '47.00', '56.58', '57.42',
        '42.08', '29.25', '42.00', '49.50', '36.75', '22.58', '27.25',
        '23.00', '27.75', '54.58', '34.17', '28.92', '29.67', '39.58',
        '56.42', '54.33', '41.00', '31.92', '41.50', '23.92', '25.75',
        '26.00', '37.42', '34.92', '34.25', '23.33', '23.17', '44.33',
        '35.17', '43.25', '56.75', '31.67', '23.42', '20.42', '26.67',
        '36.00', '25.50', '19.42', '32.33', '34.83', '38.58', '44.25',
        '44.83', '20.67', '34.08', '21.67', '21.50', '49.58', '27.67',
        '39.83', '?', '37.17', '25.67', '34.00', '49.00', '62.50', '31.42',
        '52.33', '28.75', '28.58', '22.50', '28.50', '37.50', '35.25',
        '18.67',